In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Dense, Input, LeakyReLU, Dropout
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from tqdm.auto import tqdm
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import csv

# 设置随机种子
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [2]:
# 定义训练时的评价指标
def rmse(y_true, y_pred):
  return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

In [3]:
# 递进式训练和测试
def model_train_test(epochs, batch_size, model, X_seq, Y_seq, process):
  # 保存测试损失和预测结果
  test_losses = []
  test_predictions = []
  test_actuals = []
  # 保存初始权重
  initial_weights = model.get_weights()

  for i in tqdm(range(((len(X_seq) - initial_train_size) // test_size)+1), desc=process):
    # 当前训练集和测试集
    train_end = initial_train_size + i * test_size
    test_end = train_end + test_size

    X_train, Y_train = X_seq[:train_end], Y_seq[:train_end]
    X_test, Y_test = X_seq[train_end:test_end], Y_seq[train_end:test_end]

    # 数据归一化
    scaler_X = MinMaxScaler()
    scaler_Y = MinMaxScaler()

    X_train_scaled = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)
    Y_train_scaled = scaler_Y.fit_transform(Y_train.reshape(-1, 1))

    # 使用训练集的归一化参数对测试集进行归一化
    X_test_scaled = scaler_X.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)
    Y_test_scaled = scaler_Y.transform(Y_test.reshape(-1, 1))

    # 恢复初始权重
    # model.set_weights(initial_weights)

    # 训练模型，早停
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model.fit(X_train_scaled, Y_train_scaled, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[early_stopping], verbose=0)

    # 在测试集上评估模型
    Y_pred_scaled = model.predict(X_test_scaled)
    Y_pred = scaler_Y.inverse_transform(Y_pred_scaled)
    Y_test = scaler_Y.inverse_transform(Y_test_scaled)

    # 计算RMSE
    rmse_value = round(np.sqrt(mean_squared_error(Y_test, Y_pred)),4)

    # 保存测试损失和预测结果
    test_losses.append(rmse_value)
    sum_test_loss = sum(test_losses)

  return sum_test_loss

In [4]:
# 递进式训练和测试
def model_train_test_visual(epochs, batch_size, model, X_seq, Y_seq):
  # 保存测试损失和预测结果
  test_losses = []
  test_predictions = []
  test_actuals = []
  # 保存初始权重
  initial_weights = model.get_weights()

  for i in tqdm(range(((len(X_seq) - initial_train_size) // test_size)+1)):
    # 当前训练集和测试集
    train_end = initial_train_size + i * test_size
    test_end = train_end + test_size

    X_train, Y_train = X_seq[:train_end], Y_seq[:train_end]
    X_test, Y_test = X_seq[train_end:test_end], Y_seq[train_end:test_end]

    # 数据归一化
    scaler_X = MinMaxScaler()
    scaler_Y = MinMaxScaler()

    X_train_scaled = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)
    Y_train_scaled = scaler_Y.fit_transform(Y_train.reshape(-1, 1))

    # 使用训练集的归一化参数对测试集进行归一化
    X_test_scaled = scaler_X.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)
    Y_test_scaled = scaler_Y.transform(Y_test.reshape(-1, 1))

    # 训练模型，早停
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model.fit(X_train_scaled, Y_train_scaled, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[early_stopping], verbose=0)

    # 在测试集上评估模型
    Y_pred_scaled = model.predict(X_test_scaled)
    Y_pred = scaler_Y.inverse_transform(Y_pred_scaled)
    Y_test = scaler_Y.inverse_transform(Y_test_scaled)
    Y_pred = np.round(Y_pred, 1)
    Y_test = np.round(Y_test, 1)

    # 计算RMSE
    rmse_value = round(np.sqrt(mean_squared_error(Y_test, Y_pred)),4)

    # 保存测试损失和预测结果
    test_losses.append(rmse_value)
    sum_test_loss = sum(test_losses)
    test_predictions.append(Y_pred)
    test_actuals.append(Y_test)

  return test_losses, sum_test_loss, test_predictions, test_actuals

In [5]:
# 采用递进式训练模型，初始划分，用2019年12月31日（含）之前的数据开始初始训练，用后30天数据测试，第一次测试后，将30天数据加入训练集，依次递进
initial_train_size = 6940  # 初始训练集大小
test_size = 30  # 测试集大小
time_steps = 30

# 构建时间步
# 默认采用7天时间步
def create_sequences(X,Y,timesteps=7):
    Xs, Ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        Ys.append(Y[i + time_steps])
    return np.array(Xs), np.array(Ys)

In [ ]:
# 加载数据
data = pd.read_csv('weather_data_cleaned.csv')

# 特征选择
features = ['Temp_Max', 'Temp_Avg', 'Temp_Min', 'Dew_Max', 'Dew_Avg',
       'Dew_Min', 'Humidity_Max', 'Humidity_Avg', 'Humidity_Min', 'Wind_Max',
       'Wind_Avg', 'Wind_Min', 'Pressure_Max', 'Pressure_Avg', 'Pressure_Min',
       'Temp_Max_3', 'Temp_Max_7', 'Temp_Min_3', 'Temp_Min_7', 'Temp_Avg_3',
       'Temp_Avg_7', 'Temp_Diff_Max_Avg', 'Temp_Diff_Min_Avg',
       'Temp_Diff_Max_Min', 'Dew_Max_3', 'Dew_Max_7', 'Dew_Min_3', 'Dew_Min_7',
       'Dew_Avg_3', 'Dew_Avg_7', 'Dew_Diff_Max_Avg', 'Dew_Diff_Min_Avg',
       'Dew_Diff_Max_Min', 'Humidity_Max_3', 'Humidity_Max_7',
       'Humidity_Min_3', 'Humidity_Min_7', 'Humidity_Avg_3', 'Humidity_Avg_7',
       'Humidity_Diff_Max_Avg', 'Humidity_Diff_Min_Avg',
       'Humidity_Diff_Max_Min', 'Wind_Max_3', 'Wind_Max_7', 'Wind_Min_3',
       'Wind_Min_7', 'Wind_Avg_3', 'Wind_Avg_7', 'Wind_Diff_Max_Avg',
       'Wind_Diff_Min_Avg', 'Wind_Diff_Max_Min', 'Pressure_Max_3',
       'Pressure_Max_7', 'Pressure_Min_3', 'Pressure_Min_7', 'Pressure_Avg_3',
       'Pressure_Avg_7', 'Pressure_Diff_Max_Avg', 'Pressure_Diff_Min_Avg',
       'Pressure_Diff_Max_Min']

feature_all = []
for _ in features:
    features_selected = features.copy()
    features_selected.remove(_)
    feature_all.append(features_selected)

# 定义CSV文件路径
csv_file_path = 'feature_importance.csv'

In [7]:
'''
# 筛选特征
# 读取已完成的数据
cut_data = pd.read_csv(csv_file_path)
cut = len(cut_data.index)
for _, feature in zip(features[cut:], feature_all):
    X = data[feature].values
    Y = data['target'].values
    X_seq, Y_seq = create_sequences(X, Y, time_steps)
    # GRU模型
    model = Sequential()
    model.add(Input(shape=(time_steps, X_seq.shape[2])))
    model.add(GRU(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(units=20))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss=rmse)
    # 训练模型
    sum_test_loss = model_train_test(epochs=20, batch_size=32, model=model, 
                                     X_seq = X_seq, Y_seq = Y_seq, process = _)
    sum_test_loss = round(sum_test_loss,4)
    importance = round(sum_test_loss - base_sum_test_loss,4)
    print(f'{_}重要性：{importance}')
    # 打开文件并追加数据
    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        # 写入当前特征的重要性
        writer.writerow([_, importance])
'''

"\n# 筛选特征\n# 读取已完成的数据\ncut_data = pd.read_csv(csv_file_path)\ncut = len(cut_data.index)\nfor _, feature in zip(features[cut:], feature_all):\n    X = data[feature].values\n    Y = data['target'].values\n    X_seq, Y_seq = create_sequences(X, Y, time_steps)\n    # GRU模型\n    model = Sequential()\n    model.add(Input(shape=(time_steps, X_seq.shape[2])))\n    model.add(GRU(units=50, return_sequences=True))\n    model.add(Dropout(0.2))\n    model.add(GRU(units=20))\n    model.add(Dropout(0.2))\n    model.add(Dense(1))\n    optimizer = Adam(learning_rate=0.001)\n    model.compile(optimizer=optimizer, loss=rmse)\n    # 训练模型\n    sum_test_loss = model_train_test(epochs=20, batch_size=32, model=model, \n                                     X_seq = X_seq, Y_seq = Y_seq, process = _)\n    sum_test_loss = round(sum_test_loss,4)\n    importance = round(sum_test_loss - base_sum_test_loss,4)\n    print(f'{_}重要性：{importance}')\n    # 打开文件并追加数据\n    with open(csv_file_path, mode='a', newline='') as fi

In [8]:
# base model
# 删除对模型有害的特征
X = data[features].values
Y = data['target'].values
X_seq, Y_seq = create_sequences(X, Y, time_steps)
# GRU模型
model = Sequential()
model.add(Input(shape=(time_steps, X_seq.shape[2])))
model.add(GRU(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=20))
model.add(Dropout(0.2))
model.add(Dense(1))
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=rmse)
# 训练模型
base_test_losses, base_sum_test_loss, base_test_predictions, base_test_actuals = model_train_test_visual(epochs=20, batch_size=32, model=model, 
                                 X_seq = X_seq, Y_seq = Y_seq)

  0%|          | 0/63 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━

In [9]:
print (f'baseline:{base_sum_test_loss}')
print (f'baseline_mean_loss:{round(base_sum_test_loss/63,4)}')
base_arr = np.array(base_test_losses)[np.array(base_test_losses) <= 4]
no_outliers_mean_loss = np.mean(base_arr)
print (f'baseline_no_outliers_mean_loss:{round(no_outliers_mean_loss,4)}')

baseline:134.6954
baseline_mean_loss:2.138
baseline_no_outliers_mean_loss:2.0216


In [10]:
# updated model
# 删除对模型有害的特征
feature_importance = pd.read_csv(csv_file_path)
features_updated = feature_importance.loc[feature_importance['Importance']>0, 'Feature']
X = data[features_updated].values
Y = data['target'].values
X_seq, Y_seq = create_sequences(X, Y, time_steps)

# GRU模型
model = Sequential()
model.add(Input(shape=(time_steps, X_seq.shape[2])))
model.add(GRU(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=20))
model.add(Dropout(0.2))
model.add(Dense(1))
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=rmse)

# 训练模型
updated_test_losses, updated_sum_test_loss, updated_test_predictions, updated_test_actuals = model_train_test_visual(epochs=20, batch_size=32, model=model, 
                                 X_seq = X_seq, Y_seq = Y_seq)

  0%|          | 0/63 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━

In [11]:
print (f'updated:{updated_sum_test_loss}')
print (f'updated_mean_loss:{round(updated_sum_test_loss/63,4)}')
updated_arr = np.array(updated_test_losses)[np.array(updated_test_losses) <= 4]
no_outliers_mean_loss = np.mean(updated_arr)
print (f'updated_no_outliers_mean_loss:{round(no_outliers_mean_loss,4)}')

updated:136.27020000000002
updated_mean_loss:2.163
updated_no_outliers_mean_loss:2.0349


In [12]:
flattened_test_predictions = []
for arr in base_test_predictions:
    flattened_test_predictions.extend([float(x) for sublist in arr.tolist() for x in sublist])

flattened_test_actuals = []
for arr in base_test_actuals:
    flattened_test_actuals.extend([float(x) for sublist in arr.tolist() for x in sublist])

In [40]:
data_test_result = pd.DataFrame({
    'pred': flattened_test_predictions,
    'actual': flattened_test_actuals},
    index = data[6970:]['Date'])
data_test_result['pred'] = data_test_result['pred'].apply(lambda x: round(x,1))
data_test_result.to_csv('data_test_result')

data_test_losses = pd.DataFrame([base_test_losses]).T
data_test_losses['losses'] = data_test_losses[0]
data_test_losses.drop(columns = [0], inplace = True)
data_test_losses.to_csv('data_test_losses')